In this notebook we're going to test the hypothesis that pretraining a model on domain data results in better performance of the model when finetuning a model

The three models we'll be using are:
1. FinBERT 2 v_1
2. FinBERT 2 v_2
3. DistilBERT

Note that the FinBERT's are based on the DistilBERT models
The evaluation metric we'll use are Accuracy, cross entropy loss and macro F1 average.

In [11]:
from datasets import Dataset, load_dataset, DatasetDict
from transformers import AutoModelForMaskedLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, Pipeline
from huggingface_hub import login
import os

login(os.getenv("HF_MASTER_KEY"))

In [ ]:
finetune_ds = load_dataset('takala/financial_phrasebank', 'sentences_50agree')
tokenizer_fin = AutoTokenizer.from_pretrained('Czunzun/finBERT2_v2')
tokenizer_distilbert = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

def tokenized_inputs(col, tokenizer):
    return tokenizer(col['sentence'], padding="max_length", truncation=True)

finetune_ds = finetune_ds.map(tokenized_inputs)

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

In [ ]:

finetune_ds = load_dataset('takala/financial_phrasebank', 'sentences_50agree', trust_remote_code=True)

tokenized_dataset = {}

for config in configs:
    tokenized_dataset[config['name']] = finetune_ds.map(lambda train: config['tokenizer'](train['sentence'], padding="max_length", truncation=True))

for dataset in tokenized_dataset:
    traintest_split = tokenized_dataset[dataset]['train'].train_test_split(test_size=.2)
    eval_split = traintest_split['test'].train_test_split(test_size=.5)
    ds = DatasetDict({
        'train': traintest_split['train'],
        'test': traintest_split['test'],
        'eval': eval_split['train']
    })
    tokenized_dataset[dataset] = ds


configs = [
    {'name': 'bert_base', 'model': 'google-bert/bert-base-uncased', 'tokenizer': AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')},
    {'name': 'finBERT2_v2', 'model': 'Czunzun/finBERT2_v2', 'tokenizer': AutoTokenizer.from_pretrained('Czunzun/finBERT2_v2')}
]



In [17]:
tokenized_dataset['bert_base']

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3876
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 970
    })
    eval: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3876
    })
})

In [ ]:
#Setting up the training arguments

def training_step(config):
    from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
    data_collator = DataCollatorWithPadding(tokenizer=config['tokenizer'])
    model = AutoModelForSequenceClassification.from_pretrained(config['model'])
    
    training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,

    )
    
    trainer=Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        tokenizer=config['tokenizer'],
        train_dataset=tokenized_dataset[config['name']]['train'],
        test_dataset=tokenized_dataset[config['name']]['test'],
        eval_dataset=tokenized_dataset[config['name']]['eval']
        )
    trainer.train()



def loss_metrics(results):
    

In [ ]:
#Initialize training for both datasets
for config in configs:

    #Training
    training_step(config=config)

    #computing loss metrics


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'eval'

In [ ]:
#Coimpute lo